In [1]:
'''GENERAL'''
import os
import warnings
#warnings.simplefilter('ignore')
import random
import collections
import math
import itertools
import glob
'''DATA'''
import numpy as np
import pandas as pd
'''PLOT'''
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
plt.style.use('seaborn-pastel')
font = {'size':16}
import seaborn as sns
'''STATS'''
import statsmodels.api as sm
'''SCIKIT LEARN'''
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
'''TENSORFLOW'''
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.preprocessing import text
from tensorflow.keras import utils
'''PALETTE'''
twi = '#DCB8E7' #Pale, light grayish mulberry
twi_blu = '#273873' #Dark sapphire blue
twi_pur = '#662D8A' #Moderate purple
twi_pnk = '#ED438D' #Brilliant raspberry
'''RETURN'''
!fortune | cowsay -f dragon

 _________________________________________ 
/ As the system comes up, the component   \
| builders will from time to time appear, |
| bearing hot new versions of their       |
| pieces -- faster, smaller, more         |
| complete, or putatively less buggy. The |
| replacement of a working component by a |
| new version requires the same           |
| systematic testing procedure that       |
| adding a new component does, although   |
| it should require less time, for more   |
| complete and efficient test cases will  |
| usually be available.                   |
|                                         |
| -- Frederick Brooks Jr., "The Mythical  |
\ Man Month"                              /
 ----------------------------------------- 
      \                    / \  //\
       \    |\___/|      /   \//  \\
            /0  0  \__  /    //  | \ \    
           /     /  \/_/    //   |  \  \  
           @_^_@'/   \/_   //    |   \   \ 
           //_^_/     \/_ //     |    \    \
    

## Standalone Win Prediction

In [18]:
# load pickled data
df = pd.read_pickle('data/by_team.pkl')
# split into features and label
X = df[['kill', 'assist', 'death', 'special', 'inked']]
y = df['win']

In [24]:
# make a quick model
model = Sequential()
model.add(Dense(units=5, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=5, batch_size=50)

Train on 113502 samples
Epoch 1/5
113502/113502 [==============================] - 2s 21us/sample - loss: 7.2770 - accuracy: 0.5221
Epoch 2/5
113502/113502 [==============================] - 2s 20us/sample - loss: 0.4933 - accuracy: 0.7708
Epoch 3/5
113502/113502 [==============================] - 2s 20us/sample - loss: 0.4723 - accuracy: 0.7823
Epoch 4/5
113502/113502 [==============================] - 2s 20us/sample - loss: 0.4725 - accuracy: 0.7839
Epoch 5/5
113502/113502 [==============================] - 2s 21us/sample - loss: 0.4670 - accuracy: 0.7866


In [35]:
# input user data
user = np.array([[5, 7, 1, 4, 915], [3, 1, 3, 3, 734], [9, 0, 1, 2, 663], [1, 1, 1, 1, 353]])

In [52]:
# generate score based on user data
preds= model.predict_proba(user)
(preds[:,1]*10000).astype(int).tolist()

[9923, 5729, 9998, 5135]

## Stanadlone Recommender

In [3]:
import pandas as pd
import pickle

with open('data/model.pkl', 'rb') as f:
    model = pickle.load(f)
    
scorer = pd.read_pickle('data/scorer.pkl')

def rec_weapon(weapon, modes='Turf War'):
    
    # data assignement shenanigans
    if type(weapon) == str:
        weapon = [weapon]
    if modes == 'debug':
        modes = ['Turf War', 'Splat Zones', 'Tower Control', 'Rainmaker', 'Clam Blitz']
    elif type(modes) == str:
        modes = [modes]
    
    # loop through every mode
    for mode in modes:
        # get predictions
        recs = model.get_user_recommendation(weapon, 10)
        # take predictions from scored list and sort by winrate
        recs = scorer[scorer.key.isin(recs)].sort_values(mode + '_winrate', ascending=False)
        # get the name of the highest winrate weapon
        best = recs.name.iloc[0]

        # calculate % winrate increase
        current = scorer[scorer.key.isin(weapon)].sort_values(mode + '_winrate', ascending=False).iloc[0]
        new = float(recs[mode + '_winrate'].iloc[0])
        percent = (new - float(current[mode + '_winrate'])) * 100

        # only make a recommendation if it is an improvement
        if percent <= 0:
            print ('The {} is already a great choice for {}!'.format(current['name'], mode))
        else:
            print('Try the {} for a {:.2f}% win rate increase in {}!'.format(best, percent, mode))

In [4]:
rec_weapon(['dualsweeper_custom', 'dualsweeper'], 'debug')

The Custom Dualie Squelchers is already a great choice for Turf War!
Try the Dark Tetra Dualies for a 1.77% win rate increase in Splat Zones!
Try the Clear Dapple Dualies for a 2.37% win rate increase in Tower Control!
Try the Enperry Splat Dualies for a 1.32% win rate increase in Rainmaker!
Try the Enperry Splat Dualies for a 0.46% win rate increase in Clam Blitz!
